## Implement a movie recommendation system and run it on the movie lens dataset 

(train vs test)

Measure performance on test set using RMSE (Root mean squared deviation)



In [ ]:
# First, required to compute a user-user similarity based on ratings and movies in common. 

In [ ]:
# Second, make rating predictions on the test set following the KNN idea: a prediction (user, movie) is the weighted average of other users' rating for the movie, weighted by user-similarity to the given user.